In [60]:
import os

import numpy as np
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import torch_geometric as pyg
# enable multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pyg.seed_everything(1234)

In [25]:
%qtconsole

In [26]:
%cd /Users/cgu3/Documents/Grape-Pi
%pwd

/Users/cgu3/Documents/Grape-Pi


/Users/cgu3/anaconda3/envs/python311/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/Users/cgu3/Documents/Grape-Pi'

In [61]:
import logging

import custom_graphgym # noqa, register custom modules
import torch
from torch_geometric import seed_everything
import argparse
from torch_geometric.graphgym.config import (
    cfg,
    dump_cfg,
    load_cfg,
    set_out_dir,
    set_run_dir,
)
from torch_geometric.graphgym.model_builder import create_model
from torch_geometric.graphgym.register import train_dict
from torch_geometric.graphgym.train import GraphGymDataModule, train
from torch_geometric.graphgym.utils.agg_runs import agg_runs
from torch_geometric.graphgym.utils.comp_budget import params_count
from torch_geometric.graphgym.utils.device import auto_select_device

from graphgym import logger
import shlex

parser = argparse.ArgumentParser(description='GraphGym')
parser.add_argument('--cfg', dest='cfg_file', type=str, required=True,
                    help='The configuration file path.')
parser.add_argument('--repeat', type=int, default=1,
                    help='The number of repeated jobs.')
parser.add_argument('--mark_done', action='store_true',
                    help='Mark yaml as done after a job has finished.')
parser.add_argument('opts', default=None, nargs=argparse.REMAINDER,
                    help='See graphgym/config.py for remaining options.')



# Load cmd line args
args = parser.parse_args(shlex.split('--cfg saved_results/yeast-LCQ-sageconv/config.yaml '
                                     'dataset.remove_unlabeled_data False dataset.split [1,0,0]'))
# args = parser.parse_args(shlex.split('--cfg /Users/cgu3/Documents/Grape-Pi/graphgym/configs/protein/gastric-graphsage.yaml'))
# Load config file
load_cfg(cfg, args)
# Set Pytorch environment
torch.set_num_threads(cfg.num_threads)
# Repeat for different random seeds
logger.set_printing()

seed_everything(cfg.seed)
auto_select_device()
# Set machine learning pipeline

model, datamodule = None, None
# use the right customized datamodule and graphgymmodule
if cfg.train.grape_pi == 'graphsage':
    datamodule = train_dict['graphsage_graphgym_datamodule']()
    model = train_dict['graphsage_create_model']()
    # train = train_dict['graphsage_train']
elif cfg.train.grape_pi == 'gcnconv':
    datamodule = GraphGymDataModule()
    model = train_dict['gcnconv_create_model']()
    # train = train_dict['gcnconv_train']

data_batch = next(iter(datamodule.loaders[0]))
model(data_batch)

# Print model info
logging.info(model)
logging.info(cfg)
cfg.params = params_count(model)
logging.info('Num parameters: %s', cfg.params)

model.load_state_dict(torch.load('saved_results/yeast-LCQ-sageconv/epoch=299-step=4800.ckpt')['state_dict'])

_StoreAction(option_strings=['--cfg'], dest='cfg_file', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, required=True, help='The configuration file path.', metavar=None)

_StoreAction(option_strings=['--repeat'], dest='repeat', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, required=False, help='The number of repeated jobs.', metavar=None)

_StoreTrueAction(option_strings=['--mark_done'], dest='mark_done', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Mark yaml as done after a job has finished.', metavar=None)

_StoreAction(option_strings=[], dest='opts', nargs='...', const=None, default=None, type=None, choices=None, required=True, help='See graphgym/config.py for remaining options.', metavar=None)

Rebuilding...
Done!


(tensor([[0.0789],
         [0.0690],
         [0.0809],
         ...,
         [0.0622],
         [0.0622],
         [0.0622]], grad_fn=<AddmmBackward0>),
 tensor([1, 0, 0,  ..., 0, 1, 0], dtype=torch.int32))

GraphsageGraphGymModule(
  (model): GNN(
    (encoder): FeatureEncoder()
    (pre_mp): GeneralMultiLayer(
      (Layer_0): GeneralLayer(
        (layer): Linear(
          (model): Linear(1, 10, bias=True)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (mp): GNNStackStage(
      (layer0): GeneralLayer(
        (layer): SAGEConv(
          (model): SAGEConv(10, 10, aggr=mean)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (post_mp): ExampleNodeHead(
      (layer_post_mp): MLP(
        (model): Sequential(
          (0): Linear(
            (model): Linear(10, 1, bias=True)
          )
        )
      )
    )
  )
)
accelerator: cpu
benchmark: False
bn:
  eps: 1e-05
  mom: 0.1
cfg_dest: config.yaml
custom_metrics: []
dataset:
  cache_load: False
  cache_save: False
  dir: data/yeast-LCQ
  edge_dim: 128
  edge_encoder: False
  edge_encoder_bn: True
  edge_encoder_name: Bond
  edge_message_ratio: 0

<All keys matched successfully>

In [62]:
# get the dictionary mapping from global node index to original protein accession
mapping = pd.read_csv('data/yeast-LCQ/yeast-LCQ_mapping.csv')
# create dictionary mapping from global node index to original protein accession
mapping = dict(zip(mapping['integer_id'], mapping['protein_id']))

In [63]:
# check why it is full-size batch and need to have original ID in the dataset
# how to retrieve original ID
model.eval()
accession_indices = []
all_pred_prob = []

with torch.no_grad():
    for batch in datamodule.loaders[0]:
        # batch = batch.to(cfg.accelerator)
        
        # for each batch, only use test nodes in the original mini-batch nodes
        # batch_mask = torch.cat([torch.ones(batch.batch_size), torch.zeros(len(batch.y) - batch.batch_size)], dim=0)
        # batch_mask = batch_mask.bool()
        # # mask = batch_mask & batch.test_mask
        # # raw_prob = batch.x[:, 0][mask]
        # raw_prob = batch.x[:, 0][batch_mask]
        
        logits, true = model(batch)

        # for each batch, only use test nodes in the original mini-batch nodes
        # global_node_idx = batch.n_id[mask]
        # original_id = [mapping[key] for key in global_node_idx.tolist()]
        
        logits, true, index = logits[:batch.batch_size], true[:batch.batch_size], batch.n_id[:batch.batch_size]
        logits = logits.squeeze(-1)
        pred_prob = torch.nn.functional.sigmoid(logits)
        
        accession_indices += index.tolist()
        all_pred_prob += pred_prob.tolist()
        # print(original_id)
        # print(pred_prob)
        # print(true)

GraphsageGraphGymModule(
  (model): GNN(
    (encoder): FeatureEncoder()
    (pre_mp): GeneralMultiLayer(
      (Layer_0): GeneralLayer(
        (layer): Linear(
          (model): Linear(1, 10, bias=True)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (mp): GNNStackStage(
      (layer0): GeneralLayer(
        (layer): SAGEConv(
          (model): SAGEConv(10, 10, aggr=mean)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (post_mp): ExampleNodeHead(
      (layer_post_mp): MLP(
        (model): Sequential(
          (0): Linear(
            (model): Linear(10, 1, bias=True)
          )
        )
      )
    )
  )
)

In [64]:
# convert the global node index to original protein accession
accession = [mapping[key] for key in accession_indices]

all_proteins_df = pd.DataFrame({'accession': accession, 'pred_prob': all_pred_prob})

dat = pd.read_csv('data/yeast-LCQ/raw/protein/yeast-LCQ.csv')
# combine the test_proteins_df with the original protein data
all_proteins_df = all_proteins_df .merge(dat, left_on='accession', right_on='protein', how='inner')
all_proteins_df.to_csv('/Users/cgu3/Documents/Grape-Pi/notebooks/post-training-evaluation/results'
                       '/yeast_LCQ_all_proteins_df'
                       '.csv')

# For yeast-ORBI dataset

In [65]:
import logging

import custom_graphgym # noqa, register custom modules
import torch
from torch_geometric import seed_everything
import argparse
from torch_geometric.graphgym.config import (
    cfg,
    dump_cfg,
    load_cfg,
    set_out_dir,
    set_run_dir,
)
from torch_geometric.graphgym.model_builder import create_model
from torch_geometric.graphgym.register import train_dict
from torch_geometric.graphgym.train import GraphGymDataModule, train
from torch_geometric.graphgym.utils.agg_runs import agg_runs
from torch_geometric.graphgym.utils.comp_budget import params_count
from torch_geometric.graphgym.utils.device import auto_select_device

from graphgym import logger
import shlex

parser = argparse.ArgumentParser(description='GraphGym')
parser.add_argument('--cfg', dest='cfg_file', type=str, required=True,
                    help='The configuration file path.')
parser.add_argument('--repeat', type=int, default=1,
                    help='The number of repeated jobs.')
parser.add_argument('--mark_done', action='store_true',
                    help='Mark yaml as done after a job has finished.')
parser.add_argument('opts', default=None, nargs=argparse.REMAINDER,
                    help='See graphgym/config.py for remaining options.')



# Load cmd line args
args = parser.parse_args(shlex.split('--cfg saved_results/yeast-ORBI-sageconv/config.yaml '
                                     'dataset.remove_unlabeled_data False dataset.split [1,0,0]'))
# args = parser.parse_args(shlex.split('--cfg /Users/cgu3/Documents/Grape-Pi/graphgym/configs/protein/gastric-graphsage.yaml'))
# Load config file
load_cfg(cfg, args)
# Set Pytorch environment
torch.set_num_threads(cfg.num_threads)
# Repeat for different random seeds
logger.set_printing()

seed_everything(cfg.seed)
auto_select_device()
# Set machine learning pipeline

model, datamodule = None, None
# use the right customized datamodule and graphgymmodule
if cfg.train.grape_pi == 'graphsage':
    datamodule = train_dict['graphsage_graphgym_datamodule']()
    model = train_dict['graphsage_create_model']()
    # train = train_dict['graphsage_train']
elif cfg.train.grape_pi == 'gcnconv':
    datamodule = GraphGymDataModule()
    model = train_dict['gcnconv_create_model']()
    # train = train_dict['gcnconv_train']

data_batch = next(iter(datamodule.loaders[0]))
model(data_batch)

# Print model info
logging.info(model)
logging.info(cfg)
cfg.params = params_count(model)
logging.info('Num parameters: %s', cfg.params)

model.load_state_dict(torch.load('saved_results/yeast-ORBI-sageconv/epoch=299-step=4800.ckpt')['state_dict'])

_StoreAction(option_strings=['--cfg'], dest='cfg_file', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, required=True, help='The configuration file path.', metavar=None)

_StoreAction(option_strings=['--repeat'], dest='repeat', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, required=False, help='The number of repeated jobs.', metavar=None)

_StoreTrueAction(option_strings=['--mark_done'], dest='mark_done', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Mark yaml as done after a job has finished.', metavar=None)

_StoreAction(option_strings=[], dest='opts', nargs='...', const=None, default=None, type=None, choices=None, required=True, help='See graphgym/config.py for remaining options.', metavar=None)

Rebuilding...
Done!


(tensor([[-0.0982],
         [-0.0943],
         [-0.1238],
         ...,
         [-0.0374],
         [-0.0533],
         [-0.0533]], grad_fn=<AddmmBackward0>),
 tensor([0, 0, 1,  ..., 0, 1, 1], dtype=torch.int32))

GraphsageGraphGymModule(
  (model): GNN(
    (encoder): FeatureEncoder()
    (pre_mp): GeneralMultiLayer(
      (Layer_0): GeneralLayer(
        (layer): Linear(
          (model): Linear(1, 10, bias=True)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (mp): GNNStackStage(
      (layer0): GeneralLayer(
        (layer): SAGEConv(
          (model): SAGEConv(10, 10, aggr=mean)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (post_mp): ExampleNodeHead(
      (layer_post_mp): MLP(
        (model): Sequential(
          (0): Linear(
            (model): Linear(10, 1, bias=True)
          )
        )
      )
    )
  )
)
accelerator: cpu
benchmark: False
bn:
  eps: 1e-05
  mom: 0.1
cfg_dest: config.yaml
custom_metrics: []
dataset:
  cache_load: False
  cache_save: False
  dir: data/yeast-ORBI
  edge_dim: 128
  edge_encoder: False
  edge_encoder_bn: True
  edge_encoder_name: Bond
  edge_message_ratio: 

<All keys matched successfully>

In [66]:
# get the dictionary mapping from global node index to original protein accession
mapping = pd.read_csv('data/yeast-ORBI/yeast-ORBI_mapping.csv')
# create dictionary mapping from global node index to original protein accession
mapping = dict(zip(mapping['integer_id'], mapping['protein_id']))

In [67]:
# check why it is full-size batch and need to have original ID in the dataset
# how to retrieve original ID
model.eval()
accession_indices = []
all_pred_prob = []

with torch.no_grad():
    for batch in datamodule.loaders[0]:
        # batch = batch.to(cfg.accelerator)

        # for each batch, only use test nodes in the original mini-batch nodes
        # batch_mask = torch.cat([torch.ones(batch.batch_size), torch.zeros(len(batch.y) - batch.batch_size)], dim=0)
        # batch_mask = batch_mask.bool()
        # # mask = batch_mask & batch.test_mask
        # # raw_prob = batch.x[:, 0][mask]
        # raw_prob = batch.x[:, 0][batch_mask]

        logits, true = model(batch)

        # for each batch, only use test nodes in the original mini-batch nodes
        # global_node_idx = batch.n_id[mask]
        # original_id = [mapping[key] for key in global_node_idx.tolist()]

        logits, true, index = logits[:batch.batch_size], true[:batch.batch_size], batch.n_id[:batch.batch_size]
        logits = logits.squeeze(-1)
        pred_prob = torch.nn.functional.sigmoid(logits)

        accession_indices += index.tolist()
        all_pred_prob += pred_prob.tolist()
        # print(original_id)
        # print(pred_prob)
        # print(true)

GraphsageGraphGymModule(
  (model): GNN(
    (encoder): FeatureEncoder()
    (pre_mp): GeneralMultiLayer(
      (Layer_0): GeneralLayer(
        (layer): Linear(
          (model): Linear(1, 10, bias=True)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (mp): GNNStackStage(
      (layer0): GeneralLayer(
        (layer): SAGEConv(
          (model): SAGEConv(10, 10, aggr=mean)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (post_mp): ExampleNodeHead(
      (layer_post_mp): MLP(
        (model): Sequential(
          (0): Linear(
            (model): Linear(10, 1, bias=True)
          )
        )
      )
    )
  )
)

In [68]:
# convert the global node index to original protein accession
accession = [mapping[key] for key in accession_indices]

all_proteins_df = pd.DataFrame({'accession': accession, 'pred_prob': all_pred_prob})

dat = pd.read_csv('data/yeast-ORBI/raw/protein/yeast-ORBI.csv')
# combine the test_proteins_df with the original protein data
all_proteins_df = all_proteins_df .merge(dat, left_on='accession', right_on='protein', how='inner')
all_proteins_df.to_csv('/Users/cgu3/Documents/Grape-Pi/notebooks/post-training-evaluation/results'
                       '/yeast_ORBI_all_proteins_df'
                       '.csv')